In [202]:
from detectron2 import model_zoo
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.engine import DefaultPredictor
from detectron2.utils.logger import setup_logger
from detectron2.structures import BoxMode
from detectron2.evaluation import DatasetEvaluator
import detectron2.data.transforms as T
from detectron2.config import get_cfg
import matplotlib.pyplot as plt
import numpy as np
import os
import json

In [5]:
setup_logger()
dataset_dir = "dataset/"
train_dir = dataset_dir + "train"
test_dir = dataset_dir + "test"
classes = ["red", "pink", "yellow", "blue", "orange", "green"]
camera_rows = 516
camera_cols = 656
train_ratio = 0.75

In [6]:
def get_data(img_dir):
    dataset_dicts = []
    for index, filename in enumerate([file for file in os.listdir(img_dir) if file.endswith('.json')]):
        json_file = os.path.join(img_dir, filename)
        with open(json_file) as f:
            img_label = json.load(f)

        record = {}

        img = os.path.join(img_dir, filename[:-5] + str('.jpg'))
        
        record["file_name"] = img
        record["height"] = camera_rows
        record["width"] = camera_cols
        record["image_id"] = index
        shapes = img_label["shapes"]
        objs = []
        for shape in shapes:
            px = [a[0] for a in shape['points']]
            py = [a[1] for a in shape['points']]
            poly = [(x, y) for x, y in zip(px, py)]
            poly = [p for x in poly for p in x]

            obj = {
                "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
                "bbox_mode": BoxMode.XYXY_ABS,
                "segmentation": [poly],
                "category_id": classes.index(shape['label']),
                "iscrowd": 0
            }
            objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts

In [7]:
for d in ["train", "test"]:
    DatasetCatalog.register(d, lambda d=d: get_data(dataset_dir + d))
    MetadataCatalog.get(d).set(thing_classes=classes)

In [203]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.MODEL.DEVICE = "cuda"
cfg.DATASETS.TRAIN = ("train",)
cfg.DATASETS.TEST = ("test", )
cfg.DATALOADER.NUM_WORKERS = 2
cfg.SOLVER.IMS_PER_BATCH = 3
cfg.INPUT.RANDOM_FLIP = "horizontal"
cfg.SOLVER.BASE_LR = 0.0025
cfg.SOLVER.MAX_ITER = 300
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.SOLVER.STEPS = []   
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512 
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(classes) 
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth") 

cfg.MODEL.DEVICE='cpu'
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5

predictor = DefaultPredictor(cfg)

[12/13 23:29:01 d2.checkpoint.c2_model_loading]: Following weights matched with model:
| Names in Model                                  | Names in Checkpoint                                                                                  | Shapes                                          |
|:------------------------------------------------|:-----------------------------------------------------------------------------------------------------|:------------------------------------------------|
| backbone.bottom_up.res2.0.conv1.*               | backbone.bottom_up.res2.0.conv1.{norm.bias,norm.running_mean,norm.running_var,norm.weight,weight}    | (64,) (64,) (64,) (64,) (64,64,1,1)             |
| backbone.bottom_up.res2.0.conv2.*               | backbone.bottom_up.res2.0.conv2.{norm.bias,norm.running_mean,norm.running_var,norm.weight,weight}    | (64,) (64,) (64,) (64,) (64,64,3,3)             |
| backbone.bottom_up.res2.0.conv3.*               | backbone.bottom_up.res2.0.conv3.{norm.bia

In [197]:
class Counter(DatasetEvaluator):
  def reset(self):
    self.count = 0
    self.inps = [0] * len(classes)
    self.outps = [0] * len(classes)
  def process(self, inputs, outputs):
    for input in inputs["annotations"]:
      self.inps[int(input["category_id"])] += 1
    for output in outputs:
      for i in range(len(output["instances"])):
        self.outps[int(output["instances"][i].pred_classes)] += 1

  def evaluate(self):
    # save self.count somewhere, or print it, or return it.
    out = {}
    for index, item in enumerate(classes):
      if self.inps[index] != 0:
        out[item] = self.outps[index]/self.inps[index]
    return out

In [198]:
def get_all_inputs_outputs(dataset):
  inp_data = get_data(dataset_dir + dataset)
  for index, data in enumerate(build_detection_test_loader(cfg, dataset)):
      yield inp_data[index], predictor.model(data)

In [204]:

evaluator = Counter()
evaluator.reset()
for inputs, outputs in get_all_inputs_outputs("test"):
  evaluator.process(inputs, outputs)
eval_results = evaluator.evaluate()

[12/13 23:29:05 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[12/13 23:29:05 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common.NumpySerializedList'>
[12/13 23:29:05 d2.data.common]: Serializing 14 elements to byte tensors and concatenating them all ...
[12/13 23:29:05 d2.data.common]: Serialized dataset takes 0.03 MiB


In [205]:
print(eval_results)

{'red': 0.9285714285714286, 'pink': 1.0, 'yellow': 1.2, 'blue': 0.2, 'orange': 1.0, 'green': 0.9090909090909091}


In [201]:
evaluator = COCOEvaluator("val", cfg, False, "./output/")
val_loader = build_detection_test_loader(cfg, "val")
print(inference_on_dataset(predictor.model, val_loader, evaluator))

WARNING [12/13 23:05:14 d2.evaluation.coco_evaluation]: COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.
[12/13 23:05:14 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[12/13 23:05:14 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common.NumpySerializedList'>
[12/13 23:05:14 d2.data.common]: Serializing 1 elements to byte tensors and concatenating them all ...
[12/13 23:05:14 d2.data.common]: Serialized dataset takes 0.00 MiB
[12/13 23:05:14 d2.evaluation.evaluator]: Start inference on 1 batches
[12/13 23:05:18 d2.evaluation.evaluator]: Inference done 1/1. Dataloading: 0.0000 s/iter. Inference: 3.6667 s/iter. Eval: 0.0159 s/iter. Total: 3.6826 s/iter. ETA=0:00:00
[12/13 23:05:19 d2.evaluation.evaluator]: Total inference time: 0:00:03.898423 (3.898423 s / iter per device, on 1 devices